In [1]:
from IPython.core.display import HTML, display
from string import Template
import pandas as pd
import json, random

In [2]:
HTML("<script src='d3.js'></script>")

In [3]:
js_text_template = Template('''
var data = [
    {
    "air_used": 0.742706, 
    "datestr": "2019-11-01 00:00:00", 
    "energy_used": 0.506402
  }, 
  {
    "air_used": 0.342706, 
    "datestr": "2019-12-01 00:00:00", 
    "energy_used": 0.106402
  }, 
  {
    "air_used": 0.400776, 
    "datestr": "2020-01-01 00:00:00", 
    "energy_used": 0.250462
  }, 
  {
    "air_used": 0.103541, 
    "datestr": "2020-02-01 00:00:00", 
    "energy_used": 0.040582
  }, 
  {
    "air_used": 0.97719, 
    "datestr": "2020-03-01 00:00:00", 
    "energy_used": 0.923212
  }, 
  {
    "air_used": 0.241088, 
    "datestr": "2020-04-01 00:00:00", 
    "energy_used": 0.20381
  }
];

// Margins, width and height. 
var margin = {top: 20, right: 20, bottom: 30, left: 10},
    width = 720 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

// Width of bars, without padding. 
var barRawWidth = width / (data.length + 2);
var barPadding = 10;
var xStart = barPadding + (barRawWidth/2);
var barWidth = barRawWidth - (barPadding*2);

// Scales.
var x = d3.time.scale().range([xStart, width-xStart]);
var y = d3.scale.linear().range([height, 0]);

// Construct our SVG object.
var svg = d3.select("#$maid").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

// X-axis.
var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom")
    .ticks(d3.time.month, 1)
    .tickFormat(d3.time.format('%b %Y'));
svg.append("g")
    .attr("class", "x axis")
    .attr("transform", "translate(0," + height + ")")
    //.call(xAxis);

// Date parsing.
var parseDate = d3.time.format("%Y-%m-%d %H:%M:%S");
data.forEach(function(d) {
  d.date = parseDate.parse(d.datestr);
});

// Set scale domains. 
var xExtent = d3.extent(data, function(d) { return d.date; });
var nxExtent = [d3.time.month.offset(xExtent[0], -1), d3.time.month.offset(xExtent[1], 1)];
x.domain(nxExtent);
y.domain([0, d3.max(data, function(d) { return d.air_used; })]);

// Call x-axis. 
d3.select(".x.axis")
    .transition().duration(1000)
    .call(xAxis);

// Line function.
var line = d3.svg.line()
    .x(function(d) { return x(d.date); })
    .y(function(d) { return y(d.energy_used); });

// Draw bars. 
var bars = svg.selectAll(".air_used")
        .data(data, function(d) { return d.datestr; });

bars.exit().remove();
    
bars.transition().duration(1000)
    .attr("x", function(d) { return x(d.date) - (barWidth/2); })
    .attr("width", barWidth)
    .attr("y", function(d) { return y(d.air_used); })
    .attr("height", function(d) { return height - y(d.air_used);});
    
bars.enter().append("rect")
    .attr("class", "air_used")
    .attr("width", barWidth)
    .attr("x", function(d) { return x(d.date) - (barWidth/2); })
    .attr("y", height)
    .attr("height", 0)
    .transition().duration(1000)
    .attr("y", function(d) { return y(d.air_used); })
    .attr("height", function(d) { return height - y(d.air_used);});

// Draw line. 
var linegraph = svg.selectAll("path.line").data([data], function(d) { return d.date; });
    
linegraph.attr('d', line).style("opacity", 1.0);
    
linegraph.enter().append("path")
        .attr("class", "line")
        .attr("d", line);
        
        
''');

In [4]:
js_text = js_text_template.substitute({
    "maid":"scatter-id"
})

html_template = Template('''
    <style>
    .axis path,
    .axis line,
    path.line{
      fill: none;
      stroke: #000;
    }

    .air_used {
      fill: steelblue;
    }
    </style>
    <div id="scatter-id"></div>
    <script>$js_text</script>
''')


In [5]:
HTML(html_template.substitute({'js_text':js_text}))